In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

Failed `CDLL(/Library/Frameworks/GEOS.framework/Versions/Current/GEOS)`
Failed `CDLL(/opt/local/lib/libgeos_c.dylib)`


In [2]:
#1st - read in the csv file
data = pd.read_csv("small_scale_data.csv")
#pull out only the shape column which are all in string format
shape_col = data["SHAPE"]
print (shape_col.shape)

(23,)


In [3]:
#create multigon finder and deal with it separately, then .apply(get_coord)
def find_multi(row):
    found = False
    if '), (' in row:
        found = True
    return found

In [4]:
#PREP FUNCTION COMES BEFORE FEEDING INTO THE APPROPRIATE MULTI/POLY FUNCTION TO MAKE DATA UNIFORM FORMAT
def prep_row(row):
    cleaned = row[16:-3]
    return cleaned # should look like 'lon lat, lon lat,...), (lon lat, lon lat...), (.....'

In [5]:
#GET_POLY FUNCTION CLEANS AND DEALS WITH SINGLE POLYGONS
#MULTIPOLYGON ROWS GET FED INTO GET_MULTI, WHICH THEN UTILIZES THIS FUNCTION WITHIN IT
def get_poly(data):
    # split the string into elements of a list by each comma
    vertex = data.split(",")
    # strip white spaces from each element
    vertex = [item.strip() for item in vertex ]
    # format the elements from 'latitude longitude' string to (latitude, longitude) floats
    vertex = [(float(item.split(" ")[0]) ,float(item.split(" ")[1])) for item in vertex]
    return (np.asarray(vertex))

In [6]:
#GET_MULTI DEALS WITH MULTIPOLYGONS BY UTILIZING GET_POLY
def get_multi(multi):
    #replace '), (' which separate polygons to ')&(' to be able to split properly
    fxn = multi.replace("), (", "&")
    #should look like : '(lon lat, ...)&(lon lat,...)&(lon lat,...)'
    #split polygons at the ampersand,  leaving strings ready to feed into get_poly
    fxn = fxn.split("&") 
    #should look like : ['(lon lat,...)', '(lon lat,...)', .....] 
    #create an empty 2D array to store a list of points per polygon, while still keeping together
    multi_2d = []
    #feed each polygon(i.e. each element in current fxn list) into the get_poly fxn so that each 
    #polygon has it's own formatted list of points while keeping them all together
    for i in range(len(fxn)):
        multi_2d.append(get_poly(fxn[i])) 
    multi_list = np.asarray(multi_2d)
    return multi_list.reshape((len(multi_list), 1))

In [7]:
#SCRATCH CODE THAT HAS BEEN REMOVED        

#     vertex = data.split(" ")[1:]
#     print vertex[1::2]
#     first = vertex[0].split("(")[3]
#     last = vertex[-1].split(")")[0]
#     body = vertex[1:-1].split(",")
#     vertex = [first] + vertex[1:-1] + [last]
#     print vertex
#     print map(float, vertex)
#     y = map(float, vertex)
#     y = zip(*[iter(y)]*2)
    
#     print y

#print get_coord(vertex)
# final = vertex.apply(get_coord)
# print get_coord(vertex[16])

In [8]:
#SCALE FUNCTION
def scale(num):
    #create 14 digits by padding 0's if needed
    num1 = str(abs(num[0]))
    num2 = str(abs(num[1]))
    
    if len(num1) < 15:
        num1 = num1.ljust(15, '0')
    elif len(num1) > 15:
        num1 = num1[:15]
    
    if len(num2) < 15:
        num2 = num2.ljust(15, '0')
    elif len(num2) > 15:
        num2 = num2[:15]
    
    #shift decimal for 8 decimal places
    return np.array([-1.*(float(num1) * (10**5)),-1.*(float(num2) * (10**5))])

In [9]:
#CENTER FUNCTION
def center(coord):
    # coord is an np array containing lat and long
    
    #find mean
    mean = coord.mean()
    
    #center by subtracting the mean
    coord = coord - mean
    return coord

In [10]:
#GENERATE THE GRID LINES


#FINDING RANDOM POINTS
def find_randpoints(ranges1, ranges2):
    #ranges is an array of either lat or long
    point1 = np.random.uniform(ranges1[0], ranges1[1], 4)
    point2 = np.random.uniform(ranges2[0], ranges2[1], 4)
    return point1, point2

#6th - Determine where the random points lie in which boundaries (using package)

#7th - Set a percentage and determine point location

In [11]:
#Testing
# (long upper, long lower)
coord1 = np.array([-71.13748, -71.14003])
#(lat upper, lat lower)
coord2 = np.array([42.35827, 42.3601])

lon_scaled = scale(coord1)
lon_centered = center(lon_scaled)

lat_scaled = scale(coord2)
lat_centered = center(lat_scaled)

# print find_randpoints(lon_centered, lat_centered)
# print lon_centered[0],lon_centered[1]
# print lat_centered[0], lat_centered[0]

# print find_randpoints(coord1, coord2)
rand_pt1, rand_pt2 = find_randpoints(coord1, coord2)
# print scale(rand_pt1)
scaled1 = scale(rand_pt1)
# print center(scaled1)

# print rand_pt2
# print scale(rand_pt2)
scaled2 = scale(rand_pt2)
# print center(scaled2)

In [20]:
def parse_shapes(one_row):
    prepped = prep_row(one_row)
    is_multi = find_multi(prepped)
    if is_multi == True:
        prepd_split = get_multi(prepped)
        return np.asarray((prepd_split), dtype = object)
    else:
        prepd_cleand = get_poly(prepped)
        return np.asarray((prepd_cleand), dtype = object)

In [21]:
poly_test = (parse_shapes(shape_col[10]))

In [22]:
#APPLY PARSE_SHAPES METHOD TO ALL ELEMENTS OF SHAPE_COL LIST
shape_data2 = shape_col.apply(parse_shapes)
print (shape_data2)

0     [[-71.1420930941527, 42.35814151259573], [-71....
1     [[-71.13647630362793, 42.35847749117381], [-71...
2     [[-71.14212984266281, 42.36036476249861], [-71...
3     [[-71.14150272599517, 42.36079814725054], [-71...
4     [[-71.1413093954988, 42.36061731979108], [-71....
5     [[-71.14217990894848, 42.359992040465464], [-7...
6     [[-71.14075873125084, 42.35798430346311], [-71...
7     [[-71.14195610324275, 42.36175682279947], [-71...
8     [[-71.13960333572695, 42.36151710421032], [-71...
9     [[-71.13924947993037, 42.360824484202666], [-7...
10    [[-71.13710397828213, 42.360277562649124], [-7...
11    [[-71.1409534465487, 42.361017398188416], [-71...
12    [[-71.1413093954988, 42.36061731979108], [-71....
13    [[-71.13971609398098, 42.359889649218495], [-7...
14    [[-71.13969875590529, 42.35899129723863], [-71...
15    [[-71.14224341134297, 42.36172475019508], [-71...
16    [[[[-71.14149442  42.36167523], [-71.14168146 ...
17    [[[[-71.13991272  42.36160081], [-71.13958

In [23]:
lat_rand = np.array([-71.13794487, -71.13926144, -71.13893327, -71.13919666])
lon_rand = np.array([42.35945246, 42.35927083, 42.35915658, 42.35879337])

In [35]:
ran_point0 = Point(lat_rand[0], lon_rand[0])
ran_point1 = Point(lat_rand[1], lon_rand[1])
ran_point2 = Point(lat_rand[2], lon_rand[2])
ran_point3 = Point(lat_rand[3], lon_rand[3])
polygon = Polygon(shape_data2[0])
print(polygon.contains(ran_point0))

False
(20, 2)
POLYGON ((-71.1420930941527 42.35814151259573, -71.14207254772674 42.35828984648253, -71.14200018752237 42.3584157757037, -71.14203460944461 42.35850435490381, -71.1425469318669 42.35860401301184, -71.14211924609936 42.35952220569904, -71.14230593324878 42.35956668443849, -71.14251101642689 42.35966179093536, -71.14256325588467 42.35964149429937, -71.14257717355979 42.35943475350602, -71.14272580165054 42.3592862017354, -71.14270979946581 42.35911072853421, -71.14283045998492 42.35900750827696, -71.14291368275283 42.35899933552524, -71.14303868814066 42.35906740351388, -71.14336061467044 42.3590856688587, -71.14342826498499 42.35905867737383, -71.14340701341683 42.35880199746698, -71.14348847766153 42.3583109961477, -71.1420930941527 42.35814151259573))


In [25]:
#now we want to write a loop to test point inside each of the gon, then when it's true 
#need to go back to csv file and check the type of land use attribute 



In [38]:
FUNCTION TO TEST POLYGONS
def test_point (point):
    test_pt = Point(point[0], point[1]) #make sure this works with whatever the input is 
    positive_gons = []
for i in range(len(shape_data2)):
        polygon1 = Polygon(shape_data2[i])
        in_poly = polygon.contains(test_pt)
        if in_poly:
            positive_gons.append(i)
    return positive_gons

23


ValueError: setting an array element with a sequence.

In [32]:
print (test_point([-71.13794487, 42.35945246]))

NameError: name 'shape_data' is not defined